In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar100
import numpy as np
import pandas as pd
from keras.utils import to_categorical

backend = tf.keras.backend
layers = tf.keras.layers

In [2]:
(X_train, y_train), (X_test, y_test) = cifar100.load_data()

X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

X_train, X_valid = X_train[5000:], X_train[:5000]
y_train, y_valid = y_train[5000:], y_train[:5000]

In [3]:
def identity_block(input_tensor, filters, is_training, dilation = 1):
    filters1, filters2, filters3 = filters
    
    x = layers.Conv2D(filters1, kernel_size = (1, 1), strides = (1, 1), padding = 'same')(input_tensor)
    x = layers.BatchNormalization(is_training = is_training)(x)
    x = layers.Activation('relu')(x)
    
    x = layers.Conv2D(filters2, kernel_size = (3, 3), strides = (1, 1), padding = 'same', dilation_rate = dilation)(x)
    x = layers.BatchNormalization(is_training = is_training)(x)
    x = layers.Activation('relu')(x)
    
    x = layers.Conv2D(filters3, kernel_size = (1, 1), strides = (1, 1))(x)
    x = layers.BatchNormalization(is_training = is_training)(x)
    
    x = layers.add([x, input_tensor])
    x = layers.Activation('relu')(x)
    
    return x
    
    
def conv_block(input_tensor, filters, is_training, strides = (2, 2), dilation = 1):
    filters1, filters2, filters3 = filters
    
    strides = (1, 1) if dilation > 1 else strides
    
    x = layers.Conv2D(filters1, kernel_size = (1, 1), strides = strides, padding = 'same')(input_tensor)
    x = layers.BatchNormalization(is_training = is_training)(x)
    x = layers.Activation('relu')(x)
    
    x = layers.Conv2D(filters2, kernel_size = (3, 3), strides = (1, 1), padding = 'same', dilation_rate = dilation)(x)
    x = layers.BatchNormalization(is_training = is_training)(x)
    x = layers.Activation('relu')(x)
    
    x = layers.Conv2D(filters3, kernel_size = (1, 1), strides = (1, 1))(x)
    x = layers.BatchNormalization(is_training = is_training)(x)
    
    
    shortcut = layers.Conv2D(filters3, kernel_size = (1, 1), strides = strides)(input_tensor)
    shortcut = layers.BatchNormalization(shortcut)
    
    x = layers.add([x, shortcut])
    x = layers.Activation('relu')(x)
    
    return x
    
    

def ResNet(input_tensor, params, is_training, dilation = 1):
    # 2, 3, 5, 2 -> ResNet50
    
    x = layers.ZeroPadding2D(padding=(3, 3))(input_tensor)
    x = layers.Conv2D(64, kernel_size = (7, 7), strides = (1, 1), padding = 'valid')(x)
    x = layers.BatchNormalization(is_training = is_training)(x)
    x = layers.Activation('relu')(x)
    x = layers.ZeroPadding2D(padding=(1, 1))(x)
    x = layers.MaxPooling2D((3, 3), strides=(2, 2))(x)
    
    c1 = x
    
    x = conv_block(x, 3, [64, 64, 256], strides=(1, 1))
    for i in range(params[0]):
        x = identity_block(x, 3, [64, 64, 256])
    c2 = x

    x = conv_block(x, 3, [128, 128, 512])
    for i in range(params[1]):
        x = identity_block(x, 3, [128, 128, 512])
    c3 = x

    x = conv_block(x, 3, [256, 256, 1024], dilation=dilation[0])
    for i in range(params[2]):
        x = identity_block(x, 3, [256, 256, 1024], dilation=dilation[0])
    c4 = x

    x = conv_block(x, 3, [512, 512, 2048], dilation=dilation[1])
    for i in range(params[3]):
        x = identity_block(x, 3, [512, 512, 2048], dilation=dilation[1])
    c5 = x    
    
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(100, activation = 'softmax')(x)
    
    return x

In [4]:
def Get_ResNet50(input_tensor, is_training):
    return ResNet(input_tensor, [2, 3, 5, 2], is_training)

def Get_ResNet101(input_tensor, is_training):
    return ResNet(input_tensor, [2, 3, 22, 2], is_training)

def Get_ResNet152(input_tensor, is_training):
    return ResNet(input_tensor, [2, 7, 35, 2], is_training)  